In [2]:
%pip install selenium pandas openpyxl webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install selenium webdriver-manager pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


ROUNDS_TO_COMPLETE = 10
WAIT_TIME = 15  # Tiempo de espera  para formularios dinámicos
MAX_RETRIES = 5  # Intentos máximos por operación

# Cargar datos desde Excel
try:
    df = pd.read_excel("Arena RPA FormData.xlsx")
    print("✓ Datos cargados correctamente")
except Exception as e:
    print(f" Error al cargar Excel: {e}")
    exit()

# Configurar navegador
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, WAIT_TIME)

def find_field_by_labels(field_name):
    """Busca campos usando las etiquetas visibles - más resistente a cambios"""
    label_xpaths = [
        f"//label[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{field_name.lower()}')]",
        f"//div[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{field_name.lower()}')]"
    ]
    
    for xpath in label_xpaths:
        try:
            label = driver.find_element(By.XPATH, xpath)
            # Buscar el input asociado al label
            field_id = label.get_attribute("for")
            if field_id:
                return driver.find_element(By.ID, field_id)
            
            # Si no tiene 'for', buscar input hermano
            return label.find_element(By.XPATH, "./following-sibling::input")
        except:
            continue
    
    return None

def find_field_dynamically(field_name):
    """Busca campos usando múltiples estrategias dinámicas"""
    strategies = [
        lambda: find_field_by_labels(field_name),
        lambda: driver.find_element(By.XPATH, f"//input[contains(translate(@placeholder, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{field_name.lower()}')]"),
        lambda: driver.find_element(By.XPATH, f"//input[contains(translate(@name, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{field_name.lower()}')]")
    ]
    
    for strategy in strategies:
        try:
            field = strategy()
            if field and field.is_displayed():
                return field
        except:
            continue
    
    return None

def wait_for_form_ready():
    """Espera hasta que el formulario esté listo para interactuar"""
    try:
        wait.until(lambda d: len(d.find_elements(By.XPATH, "//input[not(@type='hidden')]")) >= 5)
        time.sleep(1)  # Pausa adicional para estabilización
        return True
    except:
        return False

def process_round(round_num):
    """Procesa una ronda completa del formulario"""
    print(f"\n RONDA {round_num}/{ROUNDS_TO_COMPLETE}")
    
    # Esperar a que el formulario esté listo
    if not wait_for_form_ready():
        print(" El formulario no se cargó correctamente")
        return False
    
    # Obtener datos correspondientes (cicla si hay menos de 10 filas)
    row_data = df.iloc[(round_num-1) % len(df)]
    
    # Diccionario para mapear campos encontrados
    found_fields = {}
    
    # Identificar todos los campos necesarios
    for field_name in df.columns:
        field = find_field_dynamically(field_name)
        if field:
            found_fields[field_name] = field
        else:
            print(f" Campo no identificado: {field_name}")
    
    # Llenar los campos encontrados
    for field_name, field in found_fields.items():
        try:
            value = str(row_data[field_name])
            field.clear()
            field.send_keys(value)
            print(f"✓ {field_name}: {value}")
        except Exception as e:
            print(f" Error al llenar {field_name}: {e}")
    
    # Captura de pantalla
    driver.save_screenshot(f"capturas/round_{round_num}.png")
    print(f"📸 Captura guardada para ronda {round_num}")
    
    # Enviar formulario
    try:
        submit_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(., 'Enviar') or contains(., 'Submit')]")))
        submit_btn.click()
        print(" Formulario enviado")
        return True
    except Exception as e:
        print(f"Error al enviar formulario: {e}")
        return False

def start_next_round():
    """Maneja la transición a la siguiente ronda"""
    for attempt in range(MAX_RETRIES):
        try:
            # Esperar a que aparezca el botón de siguiente ronda
            next_btn = wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//*[contains(., 'Siguiente') or contains(., 'Next') or contains(., 'Iniciar')]")))
            next_btn.click()
            print(f" Intentando iniciar siguiente ronda (Intento {attempt+1}/{MAX_RETRIES})...")
            
            # Esperar a que cargue el nuevo formulario
            if wait_for_form_ready():
                return True
            
        except Exception as e:
            print(f" Intento {attempt+1} fallido: {str(e)}")
            time.sleep(2)
    
    return False

def main():
    try:
        os.makedirs("capturas", exist_ok=True)
        
        print("\n Iniciando el reto del formulario dinámico...")
        driver.get("https://arenarpa.com/crazy-form")
        
        # Iniciar el reto
        start_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//a[contains(., 'Iniciar') or contains(., 'Start')]")))
        start_btn.click()
        print(" Contador iniciado - ¡Comienza el reto!")
        
        # Procesar todas las rondas
        for round_num in range(1, ROUNDS_TO_COMPLETE+1):
            if not process_round(round_num):
                break
                
            if round_num < ROUNDS_TO_COMPLETE:
                if not start_next_round():
                    print(" No se pudo avanzar a la siguiente ronda")
                    break
            
            time.sleep(1)  # Pausa corta entre rondas
        
        print("\n ¡Reto completado exitosamente!")
        
    except Exception as e:
        print(f"\n Error crítico: {e}")
        driver.save_screenshot("capturas/error_final.png")
    finally:
        driver.quit()
        print("Navegador cerrado.")

if __name__ == "__main__":
    main()

✓ Datos cargados correctamente

 Iniciando el reto del formulario dinámico...
 Contador iniciado - ¡Comienza el reto!

 RONDA 1/10
✓ Nombres: Camilo
✓ Apellidos: Duarte
✓ Empresa: Base Robot
✓ Numero: 3195555555
✓ Email: camilo@duarteconsulting.com
✓ Pais: Colombia
✓ Web: Duarteconsulting.com
📸 Captura guardada para ronda 1
 Formulario enviado
 Intentando iniciar siguiente ronda (Intento 1/5)...

 RONDA 2/10
✓ Nombres: Mar
✓ Apellidos: Montes
✓ Empresa: Maremoto Datos
✓ Numero: 3205554242
✓ Email: mar@maremotodata.com
✓ Pais: Colombia
✓ Web: Maremotodata.wordpress.com
📸 Captura guardada para ronda 2
 Formulario enviado
 Intentando iniciar siguiente ronda (Intento 1/5)...

 RONDA 3/10
✓ Nombres: Yeison Andres
✓ Apellidos: Hernandez
✓ Empresa: Surtimerkar
✓ Numero: 3215551010
✓ Email: info@surtimerkar.com
✓ Pais: Peru
✓ Web: Surtimerkar.com.pe
📸 Captura guardada para ronda 3
 Formulario enviado
 Intentando iniciar siguiente ronda (Intento 1/5)...

 RONDA 4/10
✓ Nombres: Juan Felipe
✓ Ape